### If you dont want to use Wandb, disable Wandb otherwise optional

references for WANDB
https://analyticsindiamag.com/hands-on-guide-to-weights-and-biases-wandb-with-python-implementation/

https://docs.wandb.ai/


In [ ]:
!nvidia-smi

In [ ]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
!huggingface-cli login --token {hf_token}

In [ ]:
import os
os.environ["WANDB_DISABLED"]="true"

# Install required packages

In [ ]:
# !pip install transformers==4.50.3

In [ ]:
import transformers
print(transformers.__version__)

### Loading the test dataset

In [ ]:
# !pip install datasets evaluate

In [ ]:
from datasets import load_dataset
from evaluate import load
raw_datasets = load_dataset("thenlpresearcher/test_data_marathi")
metric = load("sacrebleu")

In [ ]:
import os
print(os.getcwd())

The dataset object itself is [datasetdict](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [ ]:
raw_datasets

In [ ]:
raw_datasets["test"][0]

To get a sense of how the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(raw_datasets["test"])

In [ ]:
def initialize_model(ckpt_dir, device, quantization=None):
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
    import torch

    # Quantization setup
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

#     # Load tokenizer
#     tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)

    # Load model
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    # Move to device and optionally convert to half precision
    if qconfig is None:
        model = model.to(device)
        if DEVICE == "cuda":
            model.half()

    # Make sure model is in training mode for fine-tuning
    model.eval()

    return model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig

en_indic_ckpt_dir = "indictrans2-en-indic-dist-200M-en-indic-iitb-finetuned-eng_Latn-to-mar_Deva/checkpoint-189744"
model_checkpoint = "ai4bharat/indictrans2-en-indic-dist-200M"

device = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None
en_indic_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, trust_remote_code=True)

In [ ]:
def initialize_model(ckpt_dir, device, quantization=None):
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
    import torch

    # Quantization setup
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    # Load model
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    # Move to device and optionally convert to half precision
    if qconfig is None:
        model = model.to_empty(device=device)
        if device == "cuda":
            model.half()

    # Make sure model is in training mode for fine-tuning
    model.eval()

    return model

In [ ]:
import os
print(os.getcwd())

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_path = "indictrans2-en-indic-dist-200M-en-indic-iitb-finetuned-eng_Latn-to-mar_Deva/checkpoint-189744"
bert_punct_model_final/data/Notebooks/indictrans2-en-indic-dist-200M-en-indic-iitb-finetuned-eng_Latn-to-mar_Deva/checkpoint-189744
# Load tokenizer (required)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, trust_remote_code=True)

# Load model
model = AutoModelForSeq2SeqLM.from_pretrained(model_path, trust_remote_code=True)

print("Model and tokenizer loaded successfully!")

Then we just need to pass all of this along with our datasets to the Seq2SeqTrainer:

In [ ]:
tokenizer = en_indic_tokenizer

In [ ]:
raw_datasets['test']

In [ ]:
from IndicTransToolkit.processor import IndicProcessor

In [ ]:
ip = IndicProcessor(inference=True)

In [ ]:
BATCH_SIZE = 4
src_lang, tgt_lang = "eng_Latn", "mar_Deva"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------- LOAD DATA --------------------
src_sentences = raw_datasets['test']["sent_written"]
ref_gt     = raw_datasets['test']["gt_marathi"]
ref_gem    = raw_datasets['test']["gemini_out"]
ref_cfilt  = raw_datasets['test']["cfilt_out"]

In [ ]:
print(len(src_sentences))
print(len(ref_gt))
print(len(ref_gem))
print(len(ref_cfilt))

In [ ]:
print(ip.preprocess_batch(["Chanting the choir raised…"], src_lang="eng_Latn", tgt_lang="mar_Deva"))

In [ ]:
import torch

def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip, device, batch_size=8, max_length=256):
    """
    Translate a batch of sentences using a seq2seq model like IndicTrans with safety checks.

    Args:
        input_sentences (list of str): Source sentences to translate.
        src_lang (str): Source language code, e.g., "eng_Latn".
        tgt_lang (str): Target language code, e.g., "mar_Deva".
        model: Hugging Face seq2seq model.
        tokenizer: Corresponding tokenizer.
        ip: Preprocessing object (IndicProcessor).
        device: torch device ("cuda" or "cpu").
        batch_size (int): Batch size for generation.
        max_length (int): Maximum length of generated sequence.

    Returns:
        translations (list of str): Translated sentences.
    """
    model.to(device)
    model.eval()
    translations = []

    # Safe access for decoder_start_token_id
    decoder_start_token_id = getattr(model.config, "decoder_start_token_id", None)
    pad_token_id = getattr(tokenizer, "pad_token_id", None)
    eos_token_id = getattr(tokenizer, "eos_token_id", None)

    if decoder_start_token_id is None:
        print("[Warning] decoder_start_token_id is None. Using default generation behavior.")

    for i in range(0, len(input_sentences), batch_size):
        batch = input_sentences[i : i + batch_size]
        print('here')

        # Preprocess the batch
        batch_preprocessed = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)
        if not isinstance(batch_preprocessed, list) or len(batch_preprocessed) == 0:
            print(f"[Warning] Preprocessed batch is empty at index {i}. Skipping...")
            continue

#         Debug: print first 2 sentences after preprocessing
        print(f"[Debug] Preprocessed batch sample: {batch_preprocessed[:2]}")

        # Tokenize
        inputs = tokenizer(
            batch_preprocessed,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        )

        # Move tensors to the correct device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate translations with safety parameters
        with torch.no_grad():
            try:
                generated_tokens = model.generate(
                    **inputs,
                    use_cache=True,
                    min_length=5,
                    max_length=max_length,
                    num_beams=5,
                    num_return_sequences=1,
                    early_stopping=True,
                    decoder_start_token_id=decoder_start_token_id,
                    pad_token_id=pad_token_id,
                    eos_token_id=eos_token_id
                )
            except Exception as e:
                print(f"[Error] Generation failed for batch starting at index {i}: {e}")
                continue

        # Decode generated tokens
        decoded_texts = tokenizer.batch_decode(
            generated_tokens,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

#         Debug: print first 2 decoded outputs
        print(f"[Debug] Decoded sample: {decoded_texts[:2]}")

        # Postprocess translations
        try:
            postprocessed = ip.postprocess_batch(decoded_texts, lang=tgt_lang)
            translations += postprocessed
        except Exception as e:
            print(f"[Error] Postprocessing failed for batch starting at index {i}: {e}")
            translations += decoded_texts  # fallback

        # Free GPU memory
        del inputs, generated_tokens
        torch.cuda.empty_cache()

    return translations

In [ ]:
src_lang, tgt_lang = "eng_Latn", "mar_Deva"

prefix = f"{tgt_lang} {src_lang}"

def remove_prefix(text):
    if text.startswith(prefix):
        return text[len(prefix):].strip()
    return text.strip()

In [ ]:
from evaluate import load
from tqdm import tqdm
import pandas as pd

# -------------------- TRANSLATION --------------------
valid_src = []
valid_pred = []
valid_gt = []
valid_gem = []
valid_cfilt = []

# Translate in batches
all_translations = []
for i in tqdm(range(0, len(src_sentences), BATCH_SIZE)):
    batch = src_sentences[i:i+BATCH_SIZE]
#     print(batch)
    translations = batch_translate(
        batch,
        src_lang,
        tgt_lang,
        model,
        tokenizer,
        ip,
        device=DEVICE
    )
    all_translations.extend(translations)

    if translations is None:
        print(f"[SKIPPED] Batch {i}: Returned None")
        continue

    cleaned = [remove_prefix(t) for t in translations]

    valid_src.extend(batch)
    valid_pred.extend(cleaned)
    valid_gt.extend(ref_gt[i:i + len(batch)])
    valid_gem.extend(ref_gem[i:i + len(batch)])
    valid_cfilt.extend(ref_cfilt[i:i + len(batch)])

print(f"\nSuccessful translations: {len(valid_pred)} / {len(src_sentences)}")

In [ ]:
valid_pred[2]

In [ ]:
mode = "combined"

In [ ]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": valid_src,
    "prediction": valid_pred,
    "gt": valid_gt,
    "gemini": valid_gem,
    "cfilt": valid_cfilt
})

results_df.to_csv(f"{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score, chrf_score = compute_scores(valid_pred, valid_gt, valid_gem, valid_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=valid_pred, references=[[r] for r in valid_gt])["score"],
    "Gemini": bleu.compute(predictions=valid_pred, references=[[r] for r in valid_gem])["score"],
    "CFILT":  bleu.compute(predictions=valid_pred, references=[[r] for r in valid_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to punct_{mode}_baseline_outputs_eval_metrics.txt")

In [ ]:
results_df['prediction'][0]